In [2]:
!pip install pyspark

You should consider upgrading via the '/Users/a1/Desktop/dev/test_vacancy/.venv/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from pyspark.sql import SparkSession, DataFrame

In [8]:
spark = SparkSession.builder.appName("ProductsCategories").getOrCreate()

products = spark.createDataFrame([
    (1, "Product A"),
    (2, "Product B"),
    (3, "Product C"),
    (4, "Product D")
], ["product_id", "product_name"])

categories = spark.createDataFrame([
    (1, "Category 1"),
    (2, "Category 2"),
    (3, "Category 3")
], ["category_id", "category_name"])

product_categories = spark.createDataFrame([
    (1, 1),
    (1, 2),
    (2, 1),
    (3, 3)
], ["product_id", "category_id"])

### Основной метод

In [9]:
def product_category(
        products: DataFrame,
        categories: DataFrame,
        relations: DataFrame
):
    product_category_pairs = relations.join(products, "product_id").join(categories, "category_id")
    
    product_category_pairs = product_category_pairs.select("product_name", "category_name")
    
    products_with_categories = relations.select("product_id").distinct()
    products_without_categories = products.join(products_with_categories, "product_id", "leftanti")
    
    return product_category_pairs, products_without_categories.select("product_name")
    

### Тестовые данные

In [11]:
# Примеры датафреймов
products = spark.createDataFrame([
    (1, "Product A"),
    (2, "Product B"),
    (3, "Product C"),
    (4, "Product D")
], ["product_id", "product_name"])

categories = spark.createDataFrame([
    (1, "Category 1"),
    (2, "Category 2"),
    (3, "Category 3")
], ["category_id", "category_name"])

product_categories = spark.createDataFrame([
    (1, 1),
    (1, 2),
    (2, 1),
    (3, 3)
], ["product_id", "category_id"])


In [12]:
product_category_pairs, products_without_categories = product_category(products, categories, product_categories)
print("Product-Category Pairs:")
product_category_pairs.show()

print("Products without Categories:")
products_without_categories.show()

Product-Category Pairs:


+------------+-------------+
|product_name|category_name|
+------------+-------------+
|   Product B|   Category 1|
|   Product A|   Category 1|
|   Product A|   Category 2|
|   Product C|   Category 3|
+------------+-------------+

Products without Categories:
+------------+
|product_name|
+------------+
|   Product D|
+------------+

